### Analyse der Covid-19 Fälle in Deutschland

Datenquelle: Bereinigte Daten der Johns Hopkins Unviversity

In [41]:
import pandas as pd 
import plotly.graph_objects as go 
import math

In [42]:
def plot_layout(fig, Titel):
    fig.update_layout(
        titlefont = dict(size = 36),
        title= dict(text = Titel, y = 0.95, x = 0.5, xanchor = 'center'),
        hovermode = 'x',
        autosize = False,
        width=1600, height=800, paper_bgcolor='LightSteelBlue',
        legend = dict(x = 0.05, y = 0.95, font=dict(size=18, color="black"), bgcolor="lightgray")
        )
    fig.update_xaxes(tickfont= dict(size=18))
    fig.update_yaxes(tickfont= dict(size=18))
    return fig

In [43]:
df = pd.read_csv('https://raw.githubusercontent.com/Lucas-Czarnecki/COVID-19-CLEANED-JHUCSSE/master/COVID-19_CLEAN/csse_covid_19_time_series_cleaned/time_series_covid19_cases_tidy.csv',parse_dates=['Date'])

In [44]:
df.Date.iloc[-1]

Timestamp('2020-11-22 00:00:00')

In [45]:
df.rename(columns=dict(Country_Region = 'land', Confirmed = 'infiziert', Deaths = 'tot', Recovered = 'erholt', Date = 'datum'),inplace=True)

In [46]:
df = df[df.land == 'Germany']

In [47]:
bevölkerung = 83_200_000
#bevölkerung = 10_400_000

In [48]:
# Berechnung von weiteren Daten auf der Basis der vorhandenen Informationen
df.sterberate = df.tot / df.infiziert * 100
df.aktiv = df.infiziert - df.tot - df.erholt
df.neu_infiziert = df.infiziert.diff()
df.neu_mean = df.neu_infiziert.rolling(window=6,center=True).mean()
df.proz_steig_infiziert = df.neu_infiziert / df.infiziert * 100
df.neu_aktiv = df.aktiv.diff()
df.proz_steig_aktiv = df.neu_aktiv / df.aktiv * 100
df.neu_tot = df.tot.diff()
df.tot_mean = df.neu_tot.rolling(window=6,center=True).mean()
df.proz_steig_tot = df.neu_tot / df.tot * 100
df.neu_erholt = df.erholt.diff()
df.proz_steig_erholt = df.neu_erholt / df.erholt * 100
df.beendet = df.erholt + df.tot
df.neu_beendet = df.beendet.diff()
df.sieben_tage_inzidenz = df.infiziert.diff(6)

In [49]:
df.tail()

,Province_State,land,Latitude,Longitude,datum,infiziert,tot,erholt
44365,NaN,Germany,51.165691,10.451526,2020-11-18,867484,13390,554272.0
44366,NaN,Germany,51.165691,10.451526,2020-11-19,891525,13662,570961.0
44367,NaN,Germany,51.165691,10.451526,2020-11-20,914118,13918,587780.0
44368,NaN,Germany,51.165691,10.451526,2020-11-21,927990,14061,600991.0
44369,NaN,Germany,51.165691,10.451526,2020-11-22,932367,14159,611627.0


Die Anzahl der Infizierten Personen in Deutschland ist abhängig von der Anzahl der getesteten Personen in Deutschland. Leider liegen aktuell keine verlässlichen Daten auf Tagesbasis für die Anzahl der getesteten Personen vor.

Bis zum 24.03. wurde nur als infiziert Fälle gezählt, die Positiv auf Corona Viren getestet wurden. Ab dem 25.03. werden auch Fälle als infiziert gezählt, bei denen Personen nicht getestet wurden, aber Kontakt zu positiv getesteten Personen hatten und Symptome zeigen.


In [50]:
fig = go.Figure()
fig.add_trace(go.Scatter(x=df.datum, y=df.beendet,mode='lines+markers', line=dict(color='green', width=4),name='Beendete Infektionen',stackgroup='one'))
fig.add_trace(go.Scatter(x=df.datum, y=df.aktiv,mode='lines+markers', line=dict(color='indianred', width=4),name='Aktive', fill='tonexty',stackgroup='one'))
fig.add_trace(go.Scatter(x=df.datum, y=df.infiziert,mode='lines+markers',line=dict(color='grey', width=4, dash='dot'),name='Infiziert',))
fig = plot_layout(fig, 'Summe der Infizierten in Deutschland')
fig.show()

In [51]:
fig = go.Figure()
fig.add_trace(go.Bar(x=df.datum, y=df.neu_infiziert, marker_color='grey',name='Tägliche Meldungen'))
fig.add_trace(go.Scatter(x=df.datum, y=df.neu_mean,line=dict(color='grey', width=4, dash='dot'), name='6-Tage Durchschnitt'))
fig = plot_layout(fig, 'Tägliche Neuinfektionen')
fig.show()

In [52]:
fig = go.Figure()
fig.add_trace(go.Scatter(x=df.datum, y=df.sieben_tage_inzidenz / bevölkerung * 100_000,mode='lines+markers', line=dict(color='indianred', width=4),name='7-Tage-Inzidenz'))
fig = plot_layout(fig, '7-Tage-Inzidenz je 100.000 Einwohner')
fig.show()

In [53]:
fig = go.Figure()
fig.add_trace(go.Scatter(x=df.datum, y=df.proz_steig_infiziert,line=dict(color='grey', width=4), mode='lines+markers'))
fig = plot_layout(fig, 'Steigerungsrate Infizierte in %')
fig.show()


In [54]:
p = df.proz_steig_infiziert.iloc[-1]
q = 1 + p/100
t = math.log(2) / math.log(q)
print(f'Verdoppelung der infizierten Personen alle {t:.2f} Tage')

Verdoppelung der infizierten Personen alle 148.00 Tage


In [55]:
fig = go.Figure()
fig.add_trace(go.Scatter(x=df.datum, y=df.aktiv,line=dict(color='indianred', width=3),mode='lines+markers'))
fig = plot_layout(fig, 'Summe der Aktiven Personen in Deutschland')
fig.show()

In [56]:
fig = go.Figure()
fig.add_trace(go.Scatter(x=df.datum, y=df.neu_infiziert,line=dict(color='grey', width=4),mode='lines+markers',name='neu Infizierte',))
fig.add_trace(go.Scatter(x=df.datum, y=df.neu_beendet,line=dict(color='green', width=4), mode='lines+markers', name='neu beendete Infektionen'))
fig = plot_layout(fig, 'Neu Infizierte vs. Neu beendete Infektionen')
fig.show()

In [57]:
fig = go.Figure()
fig.add_trace(go.Bar(x=df.datum, y=df.neu_aktiv, marker_color='indianred'))
fig = plot_layout(fig, 'Tägliche neue aktive Fälle')
fig.show()

In [58]:
fig = go.Figure()
fig.add_trace(go.Scatter(x=df.datum, y=df.proz_steig_aktiv,mode='lines+markers',line=dict(color='indianred', width=4)))
fig = plot_layout(fig, 'Steigerungsrate Aktive in %')
fig.show()

In [59]:
p = df.proz_steig_aktiv.iloc[-1]
q = 1 + p/100
t = math.log(2) / math.log(q)
print(f'Verdoppelung der aktiven Personen alle {t:.2f} Tage')

Verdoppelung der aktiven Personen alle -33.08 Tage


In [60]:
fig = go.Figure()
fig.add_trace(go.Scatter(x=df.datum, y=df.tot,mode='lines+markers',line=dict(color='indianred', width=4)))
fig = plot_layout(fig, 'Summe der verstorbenen Personen in Deutschland')
fig.show()

In [61]:
fig = go.Figure()
fig.add_trace(go.Bar(x=df.datum, y=df.neu_tot,marker_color='indianred', name='Tägliche Meldung'))
fig.add_trace(go.Scatter(x=df.datum, y=df.tot_mean,line=dict(color='indianred', width=4, dash='dot'), name='6-Tage Durchschnitt'))
fig = plot_layout(fig, 'Täglich Verstorbene')
fig.show()

In [62]:
fig = go.Figure()
fig.add_trace(go.Scatter(x=df.datum, y=df.proz_steig_tot,mode='lines+markers',line=dict(color='indianred', width=4)))
fig = plot_layout(fig, 'Steigerungsrate Sterbefälle in %')
fig.show()

In Deutschland gilt jeder Verstorbene, bei dem Corona-Viren gefunden wurden als Corona Sterbefall, unabhängig davon, ob er an Corona verstorben ist.

Update 08.04.2020: Die Gesundheitsbehörde der Stadt Hamburg erklärte jüngst, die Corona-Todesfälle anders zu zählen als das RKI. Beim Institut landen alle Verstorbenen, bei denen das Virus SARS-CoV-2 festgestellt wurde, in die Todeszählung. In Hamburg nur diejenigen, die nachweislich auch an Covid-19 gestorben sind. Das untersucht wiederum die Rechtsmedizin: Bei Sterbefällen mit positivem Corona-Test wird bei einer Obduktion die genaue Todesursache festgestellt. https://www.merkur.de/welt/coronavirus-deutschland-rki-zahlen-statistiken-falsch-tote-covid-19-robert-koch-institut-kritik-zweifel-zr-13640817.html



In [63]:
print(f'Sterberate in Deutschland aktuell bei {df.sterberate.iloc[-1]:.2f} %')

Sterberate in Deutschland aktuell bei 1.52 %


In [64]:
fig = go.Figure()
fig.add_trace(go.Scatter(x=df.datum, y=df.erholt / df.beendet * 100,line=dict(color='green', width=4),mode='lines+markers',name='erholt'))
fig.add_trace(go.Scatter(x=df.datum, y=df.tot / df.beendet * 100,line=dict(color='indianred', width=4), mode='lines+markers', name='verstorben'))
fig = plot_layout(fig, 'Verteilung beendete Fälle in %')
fig.show()

# Sterblichkeit je Altersklassen

Leider gibt es aktuell keine verlässlichen Daten zur Sterblickheit in Deutschland je Altersklassen. Auf der Suche nach Daten bin ich schließlich auf Süd-Korea gestoßen. Süd-Korea ist insofern interessant für Deutschland, da der Zustand des Gesundheitssystem in etwas auf dem Stand von Deutschland ist und dort genaue Daten zur Corona-Krise verfügbar sind.

Lt. Bericht von Professor Kim Woo-joo des Guro-Krankenhauses der Korea-Universität vom 28.03.2020 (https://www.youtube.com/watch?v=gAk7aX5hksU&t=284s) liegt die Sterbewahrscheinlichkeit für die unterschiedlichen Altersgruppen ins Südkorea bei folgenden Werten:

0-9 Jahre 0%, 10 - 19 Jahre 0%, 20 - 29 Jahre 0%, 30 - 39 Jahre 0,1%, 40 - 49 Jahre 0,1%, 50 - 59 Jahre 0,5%, 60 - 69 Jahre 1,6%, 70 - 79 Jahre 6.3%, ab 80 Jahre 11.6%

Aus der Bevölerkerungspyramide Deutschland werden die Anzahl Personen je Altersklasse dagegengesetzt (in Tausend):
0-9 Jahre 7760, 10-19 Jahre 7537, 20-29 Jahre 9573, 30-39 Jahre 10940, 40-49 Jahre 10105, 50-59 Jahre 13331, 60-69 Jahre 10741, 70 - 79 Jahre 7456, 80-89 Jahre 5074, 90-99 Jahre 832

Bei sofortiger uneingeschränkter Bewegungsfreiheit ohne besonderen Schutz der gesamten Bevölkerung würden sich folgende Todesfälle ergeben


In [65]:
risikoklassen = dict(
    altersklasse = "0-9,10-19,20-29,30-39,40-49,50-59,60-69,70-79,80-89,90-99".split(','),
    anzahl =  [7760, 7537, 9573, 10940, 10105, 13331, 10741, 7456, 5074, 832],
    sterblichkeit = [0, 0, 0, 0.1, 0.1, 0.5, 1.6, 6.3, 11.6, 11.6])
df_alter = pd.DataFrame.from_dict(risikoklassen)
df_alter.sterbefälle = df_alter.anzahl * df_alter.sterblichkeit / 100 * 1000
fig = go.Figure()
fig.add_trace(go.Bar(x=df_alter.altersklasse, y=df_alter.sterbefälle, text=df_alter.sterbefälle))
fig = plot_layout(fig, 'Anzahl möglicher Tote je Altersklasse')
fig.update_traces(texttemplate='%{text:,.0f}', textposition='outside')
fig.show()

### Prognose der benötigten Intensivbetten

* Basis für die Prognose ist die Anzahl der aktiven Fälle.
* Das Verhältnis zwischen erholt/verstorben bei den beendeten Fällen liegt ca. bei 95%/5%
* Damit liegt der Anteil der benötigten Intensivbetten mind. bei 5% der aktiven Fälle
* Da die Krankheit Zeit benötigt, um sich zu einem kritischen Fall zu entwickeln wird ein Zeitverzug von 10 Tagen ab Infektionserkennung berücksichtigt
* Angenommen wird eine Behandlungszeit von 8 Tagen auf der Intensivstation, danach wird das Bett wieder frei


In [66]:
anteil_intensiv = df.tot.iloc[-1] / df.beendet.iloc[-1] * 100
df.intensivbetten = df.aktiv.shift(10) * anteil_intensiv / 100
df.freie_betten = df.intensivbetten.shift(8)
fig = go.Figure()
fig.add_trace(go.Scatter(x=df.datum, y=df.intensivbetten - df.freie_betten,line=dict(color='indianred', width=4),mode='lines+markers'))
fig = plot_layout(fig, 'Prognose der benötigten Intensivbetten')
fig.show()


lt. Melderegister DIVI liegt die Anzahl der Intensiv behandelten Covid-Patienten in den Kliniken am 03.04.2020 17:22 Uhr bei 1.119 Patienten (s. https://www.divi.de/images/Dokumente/DIVI-IntensivRegister_Tagesreport_2020_04_04.pdf ). Dies würde gut zu der dargestellten Kurve mit den dahinterliegenden Annahmen passen.